# cumulative answered_correctly with question tags

In [ ]:
lst = []
for t in range(188):
    s = f"""
  CASE WHEN q_tag_{t}=1 AND content_type_id=0
    THEN ROW_NUMBER() OVER (PARTITION BY user_id, q_tag_{t}, content_type_id ORDER BY row_id) END
    AS work_q_tag_{t}_v1,"""
    lst.append(s)
work_q_tag_v1 = "".join(lst)

lst = []
for t in range(188):
    s = f"""
  SUM(
    CASE WHEN q_tag_{t}=1 AND content_type_id=0
    THEN answered_correctly END
    ) OVER(PARTITION BY user_id, q_tag_{t}, content_type_id ORDER BY row_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    AS cumsum_q_tag_{t}_v1,"""
    lst.append(s)
cumsum_q_tag_v1 = "".join(lst)

lst = []
for t in range(188):
    s = f"""
  IFNULL(LAST_VALUE(work_q_tag_{t}_v1 IGNORE NULLS) OVER(PARTITION BY user_id ORDER BY row_id), 0) AS work_q_tag_{t}_v2,"""
    lst.append(s)
work_q_tag_v2 = "".join(lst)

lst = []
for t in range(188):
    s = f"""
  IFNULL(LAST_VALUE(cumsum_q_tag_{t}_v1 IGNORE NULLS) OVER(PARTITION BY user_id ORDER BY row_id), 0) AS cumsum_q_tag_{t}_v2,"""
    lst.append(s)
cumsum_q_tag_v2 = "".join(lst)

lst = []
for t in range(188):
    s = f"""
  IFNULL(LAG (work_q_tag_{t}_v2, 1) OVER (PARTITION BY user_id ORDER BY row_id), 0) AS work_q_tag_{t}_v3,"""
    lst.append(s)
work_q_tag_v3 = "".join(lst)


lst = []
for t in range(188):
    s = f"""
  IFNULL(LAG (cumsum_q_tag_{t}_v2, 1) OVER (PARTITION BY user_id ORDER BY row_id), 0) AS cumsum_q_tag_{t}_v3,"""
    lst.append(s)
cumsum_q_tag_v3 = "".join(lst)

In [ ]:
query = f"""
CREATE TABLE Kaggle_Riiid.create_feat_cumsum_q_tag AS
SELECT
  row_id,
  -- work_q_tag{work_q_tag_v3}
  -- correct_q_tag{cumsum_q_tag_v3}
FROM (
SELECT
  row_id, user_id,
  -- work_q_tag{work_q_tag_v2}
  -- correct_q_tag{cumsum_q_tag_v2}
FROM (
SELECT
  row_id, user_id,
  -- work_q_tag{work_q_tag_v1}
  -- correct_q_tag{cumsum_q_tag_v1}
FROM Kaggle_Riiid.train
LEFT JOIN Kaggle_Riiid.question_tags_ohe ON question_id = content_id
))
"""
print(query)

In [ ]:
lst = []
for t in range(188):
    s = f"""
  IFNULL(SAFE_DIVIDE(cumsum_q_tag_{t}_v3, work_q_tag_{t}_v3), 0) AS correct_rate_q_tag_{t},"""
    lst.append(s)
correct_rate_q_tag = "".join(lst)

In [ ]:
query = f"""
CREATE TABLE Kaggle_Riiid.create_feat_rate_q_tag AS
SELECT
  row_id, {correct_rate_q_tag}
FROM Kaggle_Riiid.create_feat_cumsum_q_tag
"""
print(query)

# lecture PCA features

In [ ]:
lst = []
for t in range(188):
    s = f"""
  SUM(
    CASE WHEN l_tag_{t}=1 AND content_type_id=1 THEN 1 END
    ) OVER(PARTITION BY user_id, l_tag_{t} ORDER BY row_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
    AS work_l_tag_{t}_v1,"""
    lst.append(s)
work_l_tag_v1 = "".join(lst)

lst = []
for t in range(188):
    s = f"""
  IFNULL(LAST_VALUE(work_l_tag_{t}_v1 IGNORE NULLS) OVER(PARTITION BY user_id ORDER BY row_id), 0) AS work_l_tag_{t}_v2,"""
    lst.append(s)
work_l_tag_v2 = "".join(lst)

In [ ]:
query = f"""
CREATE TABLE Kaggle_Riiid.create_feat_cumsum_l_tag AS
SELECT
  row_id, user_id,{work_l_tag_v2}
FROM(
SELECT
  row_id, user_id,{work_l_tag_v1}
FROM Kaggle_Riiid.train 
LEFT JOIN Kaggle_Riiid.lecture_tags_ohe ON lecture_id = content_id
)
"""
print(query)

# for infer infomation

In [ ]:
lst = []
for t in range(188):
    s = f"""
  IFNULL(SUM(CASE WHEN content_type_id=0 AND q_tag_{t}=1 THEN 1 END), 0) AS q_tag_{t}_work,"""
    lst.append(s)
q_tag_work_agg_user = "".join(lst)

lst = []
for t in range(188):
    s = f"""
  IFNULL(SUM(CASE WHEN content_type_id=0 AND q_tag_{t}=1 AND answered_correctly=1 THEN 1 END), 0) AS q_tag_{t}_correct,"""
    lst.append(s)
q_tag_correct_agg_user = "".join(lst)

lst = []
for t in range(188):
    s = f"""
  IFNULL(SUM(CASE WHEN content_type_id=1 AND l_tag_{t}=1 THEN 1 END), 0) AS l_tag_{t}_work,"""
    lst.append(s)
l_tag_work_agg_user = "".join(lst)

In [ ]:
query = f"""
CREATE TABLE Kaggle_Riiid.user_agg_info AS
SELECT
  user_id, 
  -- q_tag work{q_tag_work_agg_user}
  -- q_tag correct{q_tag_correct_agg_user}
  -- l_tag wok{l_tag_work_agg_user}
FROM Kaggle_Riiid.train
LEFT JOIN Kaggle_Riiid.question_tags_ohe ON question_id = content_id
LEFT JOIN Kaggle_Riiid.lecture_tags_ohe ON lecture_id = content_id
GROUP BY user_id"""
print(query)

# for train ignore

In [ ]:
lst = []
for i in range(188):
    s = f"""
                 cumsum_q_tag_{i}_v3,"""
    lst.append(s)
cumsum_q_tag = "".join(lst)
print(cumsum_q_tag)

# avairable tag

In [ ]:
lst = []
for t in range(188):
    s = f"correct_rate_q_tag_{t} * q_tag_{t}"
    lst.append(s)
rate_x_tag = " +\n    ".join(lst)

lst = []
for t in range(188):
    s = f"q_tag_{t}"
    lst.append(s)
n_tag = " +\n    ".join(lst)

In [ ]:
query = f"""
CREATE TABLE Kaggle_Riiid.create_feat_available_tag_rate AS
SELECT
  row_id,
  IFNULL(SAFE_DIVIDE(
    {rate_x_tag}
  ,
    {n_tag}
  ), 0) AS q_tag_rate_this_avg
FROM  Kaggle_Riiid.train
LEFT JOIN  Kaggle_Riiid.create_feat_rate_q_tag USING (row_id)
LEFT JOIN  Kaggle_Riiid.question_tags_ohe ON train.content_id = question_tags_ohe.question_id
"""
print(query)